## Problem: 

An airline wants to know the minimum number of flight routes to add such that any given airport can connect to any other airport.

Given the following 3 inputs ...
- a list of one way airport routes [origin, destination] --> <b><i>routes</i></b>
- list of all airports --> <b><i>airports</i></b>
- a starting airport --> <b><i>startingAirport</i></b>

complete the function <u><b>minAdditionalRoutes</b></u> to return a list with 2 elements:
1.  first element is an integer representing the MINIMUM number of additional routes to add to the existing one way <b><i>routes</i></b> list such that a given <b><i>startingAirport</i></b> airport location can travel to <u>any</u> any other airport location in the list <b><i>airports</i></b>.
2.  second element is a sorted list of which destination airports from the <b><i>startingAirport</i></b> to add to meet the condition above.  If sets of airports are interchangeable to add (e.g. highly connected components), group them together in their own sorted list (e.g. ['EYW', 'LHR', 'SAN', 'SFO'])


In [1]:
def minAdditionalRoutes(startingAirport, routes, airports):
    # function returns a recursive list (paths) of airports 
    # from "start" airport
    # using a 1st degree adjacency path dictionary
    def findRecursivePaths(start, path_dict, \
                           paths=set(), \
                           traversed=set()):
        
        if start not in path_dict:
            return paths
        
        paths.add(start)
        
        _paths = [a for a in path_dict[start]]
        
        for _path in _paths:
            # stop when path has already been traversed
            if tuple([start, _path]) in traversed:
                break
            else:
                traversed.add(tuple([start, _path]))
            paths.add(_path)
            findRecursivePaths(_path, path_dict, \
                               paths=paths, \
                               traversed=traversed)
        return paths
            
    min_paths = 0
    paths_to_add = []
    
    # deduplicate
    airports = set(airports)
    routes = list(set(tuple(row) for row in routes))
    
    # base case
    if startingAirport not in airports:
        return 'startingAirport is not in airports'
    
    # create dicts of 1st-degree adjacent upstream/downstream paths
    upstream_dict, downstream_dict = dict(), dict()
    for d in airports:
        upstream_dict[d] = set()
        downstream_dict[d] = set()
        for r in routes:
            if r[1] == d:
                upstream_dict[d].add(r[0])
            if r[0] == d:
                downstream_dict[d].add(r[1])
    
    # recursively find all upstream and downstream paths from all airports
    r_upstream_dict, r_downstream_dict = dict(), dict()
    r_downstream_inverse_dict = dict()
    for airport in airports:
        upstream_paths = findRecursivePaths(airport, 
                                            upstream_dict, 
                                            paths=set(),
                                            traversed=set())
        
        upstream_paths = sorted(list(upstream_paths))

        r_upstream_dict[airport] = upstream_paths

        downstream_paths = findRecursivePaths(airport, 
                                              downstream_dict, 
                                              paths=set(),
                                              traversed=set())
    
        downstream_paths = sorted(list(downstream_paths))
        r_downstream_dict[airport] = downstream_paths
        if str(downstream_paths) not in r_downstream_inverse_dict:
            r_downstream_inverse_dict[str(downstream_paths)] = [airport]
        else:
            r_downstream_inverse_dict[str(downstream_paths)].append(airport)
            r_downstream_inverse_dict[str(downstream_paths)].sort()
    
    # find missing recursive downstream connections from startingAirport
    missing = set(airports).difference(set(r_downstream_dict[startingAirport]))
    
    # base case
    if len(missing) == 0:
        return [min_paths, sorted(paths_to_add)]
    
    # add airports with no upstream connections 
    for k,v in r_upstream_dict.items():
        if v == [k]:
            min_paths += 1
            paths_to_add.append([k])
            missing.remove(k)
            missing = missing.difference(set(r_downstream_dict[k]))
    
    # add the rest of the airports with most downstream connections 
    # first and keep adding the downstream connections until there are no 
    # missing recursive downstream connections from startingAirport
    downstream_left = [r_downstream_dict[m] for m in missing]
    
    downstream_sets = [list(y) for y in set(tuple(x) for x in downstream_left)]
    downstream_sets.sort(key=lambda x:len(x), reverse=True)
    
    for downstream_set in downstream_sets:
        min_paths += 1
        missing = missing.difference(set(downstream_set))
        paths_to_add.append(r_downstream_inverse_dict[str(downstream_set)])
        if len(missing) == 0:
            break
            
    return [min_paths, sorted(paths_to_add)]



In [2]:
# first set of tests
routes = [
    ['DSM', 'ORD'],
    ['ORD', 'BGI'],
    ['BGI', 'LGA'],
    ['SIN', 'CDG'],
    ['CDG', 'SIN'],
    ['CDG', 'BUD'],
    ['DEL', 'DOH'],
    ['DEL', 'CDG'],
    ['TLV', 'DEL'],
    ['EWR', 'HND'],
    ['HND', 'ICN'],
    ['HND', 'JFK'],
    ['ICN', 'JFK'],
    ['JFK', 'LGA'],
    ['EYW', 'LHR'],
    ['LHR', 'SFO'],
    ['SFO', 'SAN'],
    ['SFO', 'DSM'],
    ['SAN', 'EYW']
]


airports = ['BGI', 'CDG', 'DEL', 'DOH', 'DSM', 'EWR', 'EYW', 'HND', 'ICN', 
            'JFK', 'LGA', 'LHR', 'ORD', 'SAN', 'SFO', 'SIN', 'TLV', 'BUD']


# tests
assert minAdditionalRoutes('LGA', routes, airports) == \
[3, [['EWR'], ['EYW', 'LHR', 'SAN', 'SFO'], ['TLV']]]
assert minAdditionalRoutes('EYW', routes, airports) == \
[2, [['EWR'], ['TLV']]]
assert minAdditionalRoutes('DSM', routes, airports) == \
[3, [['EWR'], ['EYW', 'LHR', 'SAN', 'SFO'], ['TLV']]]
assert minAdditionalRoutes('SFO', routes, airports) == \
[2, [['EWR'], ['TLV']]]
assert minAdditionalRoutes('XXX', routes, airports) == \
'startingAirport is not in airports'

In [3]:
# second set of tests

routes2 = [
    ['LAS', 'LAX'],
    ['LAX', 'LAS'],
    ['LGA', 'LAX'],
    ['LAX', 'LGA'],
    ['LGA', 'BOS'],
    ['ATL', 'ORD'],
    ['ORD', 'ATL'],
    ['ORD', 'SEA'],
    ['MIA', 'SAN'],
    ['SAN', 'MIA'],
    ['SFO', 'SAN'],
    ['SFO', 'SAN'],
    ['MIA', 'FLL'],
    ['FLL', 'SFO'],
    ['FLL', 'DFW'],
    ['DFW', 'BOS']
]


airports2 = ['LAS', 'LAX', 'LGA', 'BOS', 'ATL', 'ORD', 'SEA', 'MIA', 'SAN', 
            'SFO', 'FLL', 'DFW', 'BOS']

# 
assert minAdditionalRoutes('BOS', routes2, airports2) == \
[3, [['ATL', 'ORD'], ['FLL', 'MIA', 'SAN', 'SFO'], ['LAS', 'LAX', 'LGA']]]
assert minAdditionalRoutes('SFO', routes2, airports2) == \
[2, [['ATL', 'ORD'], ['LAS', 'LAX', 'LGA']]]
assert minAdditionalRoutes('LGA', routes2, airports2) == \
[2, [['ATL', 'ORD'], ['FLL', 'MIA', 'SAN', 'SFO']]]
assert minAdditionalRoutes('SEA', routes2, airports2) == \
[3, [['ATL', 'ORD'], ['FLL', 'MIA', 'SAN', 'SFO'], ['LAS', 'LAX', 'LGA']]]
assert minAdditionalRoutes('TLV', routes2, airports2) == \
'startingAirport is not in airports'
